# 0. 크롤링

In [ ]:
# 국민청원 홈페이지 개정으로 쓸 수 없는 크롤링 코드
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

# 크롤링한 데이터를 저장할 빈 데이터 프레임
result = pd.DataFrame()
# 국민청원 글 URL 끝 숫자를 통한 for문
for i in range(584274, 595226):
    URL = "http://www1.president.go.kr/petitions/" + str(i)
    
    # URL로 get 요청(request)을 보내 그 응답을 'response'에 저장 
    response = requests.get(URL)
    # F12 눌렀을 때 나오는 HTML 코드 저장
    html = response.text
    # HTML 코드 파싱
    soup = BeautifulSoup(html, 'html.parser')
    # find 함수를 통해 내가 원하는 값 가져오기
    title = soup.find('h3', class_ = 'petitionsView_title')
    count = soup.find('span', class_ = 'counter')
    
    for content in soup.select('div.petitionsView_write > div.View_write'):
        # 청원 내용
        content
    
    a = []
    for tag in soup.select('ul.petitionsView_info_list > li'):
        # 청원정보
        a.append(tag.contents[1])
        
    if len(a) != 0:
        df1 = pd.DataFrame({
            # 청원시작일
            'start' : [a[1]],
            # 청원마감일
            'end' : [a[2]],
            # 카테고리
            'category' : [a[0]],
            # 참여인원
            'count' : [count.text],
            # 청원 제목
            'title' : [title.text],
            # 청원내용
            # 13000자 로 제한
            # strip() : 앞뒤 공백 제거
            'content' : [content.text.strip()[0:13000]]
        })
        # DataFrame에 누적 병합
        result = pd.concat([result, df1])
        # index 부여
        result.index = np.arange(len(result))
        
    # 웹페이지 정책에 따라 60건당 90초 대기
    if i % 60 == 0:
        # 상태 출력
        print('Sleep 90 seconds. Count : ' + str(i)
              + ', Local Time : '
             + time.strftime('%Y - %m - %d', time.localtime(time.time()))
             + ' ' + tiem.strftime('%X', time.localtime(time.time()))
             + ', Data Length : ' + str(len(result)))
        time.sleep(90)

### 0-0. 크롤링 데이터 확인

In [ ]:
print(result.shape)

In [ ]:
df = result
df.head

In [ ]:
# csv 파일로 저장
df.to_csv('data/crawling.csv', index = False, encoding = 'utf-8-sig')

# -------------------------------------------------------------------------------------------

# 1. 데이터 전처리

In [1]:
# 크롤링이 불가하므로, 다운로드 받은 자료 사용
# https://github.com/bjpublic/DeepLearningProject
# https://github.com/deep-learning-with-projects/deep-learning-with-projects

import pandas as pd
df = pd.read_csv('./crawling.csv')
df.head(5)

,category,content,count,end,start,title
0,인권/성평등,본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 ...,267,2020-02-01,2020-01-02,서울지방병무청 탈의실에 설치된 CCTV에 대한 진상규명을 요구한다. 또한 인권위의 ...
1,경제민주화,우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저...,271,2020-02-01,2020-01-02,주식시장 활성화 및 소액(개미)투자자 보호
2,행정,억울한 일로 국민청원을 신청합니다.\r\n\r\r\n 저는 **구치소 **교도관입니...,198,2020-02-01,2020-01-02,교정기관의 민낮
3,안전/환경,미세먼지의 심각성은 이제 적극적인 대안을 요구 하고 있습니다.\r\r\n우리 일상에...,170,2020-02-01,2020-01-02,미세먼지 저감 대책
4,교통/건축/국토,저는 우선 아이셋의 부모입니다.\r\r\n식구가 많은편이고 아이들이 성장함에 따라 ...,"2,127",2020-02-01,2020-01-02,악질세입자 방지를 위한 세입자보호법을 재정해주세요.


In [2]:
# content 내용 확인
# \r, \n 등 엔터나 탭 등의 공백문자 제거 필요
df.loc[0]['content']

'본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 발견하였다. 탈의실 천장에 CCTV를 발견한 것이다. 본인은 이에 경악하였다.\r\n\r\r\n탈의실에 CCTV를 설치하는 것은 개인정보보호법 25조 2항을 위반한 사안으로 심각한 법적 문제이다. 촬영이 되었다면 이는 성폭력범죄의 처벌 등에 관한 특례법 14조 1항 위반이기도 하다. 또한, 법적인 문제일 뿐만 아니라 개인의 자유를 침해하고 성적 수치심을 불러일으키는 매우 심각한 윤리적 문제이다.\r\r\n이렇듯 불법카메라는 그 자체로 굉장히 심각한 문제이다. 그런데 공공기관에서 탈의실에 불법, 비윤리적으로 카메라를 설치하였고, 이를 운영하였을지도 모르는 상황이 발생하였다. 도대체 한국 정부는 인권을 무엇이라고 생각하는 것인가? \r\n\r\r\n발견 직후 인권위에 진정을 넣었지만 인권위에서는 현재까지 조사가 되고 있는지, 진행은 어떻게 되었는지 어떠한 답변도 하지 않고 있다. 병무청측에서도 어떠한 답변도 없었다.\r\n\r\r\n더 이상 답변을 기다리는 것이 의미 없다고 생각한 본인은 모 커뮤니티에 이 사실을 알렸다. 이후 이슈화가 되자 병무청은 설치는 되었으나 운영은 하지 않았다는 대답을 내놓았지만 이에 대한 조사 경과 발표나 사과, 책임 있는 대응은 전혀 보여주고 있지 않다.\r\n\r\r\n이는 개인의 자유와 권리의 본질적 내용을 침해하는 강제징집, 국제노동기구가 인정한 강제징용/강제노동을 당하는 20대 남성들이 국가에 의해 신체를 감시당한 사건이다. 한국 정부는 병역의무자에게는 인권이 없다고 생각하는가?\r\n\r\r\n언제까지 국가는 개인들을 노예 취급할 것인가? 몇백원도 안되는 시급을 주며 강제로 폭력 속에서 부리는 것도 모자라 이제는 이런 어처구니 없는 일까지 일어나는 것에 경악하지 않을 수 없다. \r\n\r\r\n이에 본인은 위 사실을 강력히 규탄하며 아래와 같은 대응을 요구한다.\r\n\r\r\n1. 서울지방병무청 징병검사소 탈의실에 CCTV가 설치된 경위와 

### - 1. 전처리

In [3]:
# 정규 표현식 지원 모듈
import re

# 공백 문자 제거 함수 정의
def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

# 특수문자 제거 함수 정의
def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ 가-힣 0-9]', ' ', str(text))
    return text

# 청원 제목 함수 적용
df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)
# 청원 내용 함수 적용
df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [4]:
# 적용 확인
df.loc[0]['content']

'본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 발견하였다  탈의실 천장에     를 발견한 것이다  본인은 이에 경악하였다      탈의실에     를 설치하는 것은 개인정보보호법 25조 2항을 위반한 사안으로 심각한 법적 문제이다  촬영이 되었다면 이는 성폭력범죄의 처벌 등에 관한 특례법 14조 1항 위반이기도 하다  또한  법적인 문제일 뿐만 아니라 개인의 자유를 침해하고 성적 수치심을 불러일으키는 매우 심각한 윤리적 문제이다    이렇듯 불법카메라는 그 자체로 굉장히 심각한 문제이다  그런데 공공기관에서 탈의실에 불법  비윤리적으로 카메라를 설치하였고  이를 운영하였을지도 모르는 상황이 발생하였다  도대체 한국 정부는 인권을 무엇이라고 생각하는 것인가       발견 직후 인권위에 진정을 넣었지만 인권위에서는 현재까지 조사가 되고 있는지  진행은 어떻게 되었는지 어떠한 답변도 하지 않고 있다  병무청측에서도 어떠한 답변도 없었다      더 이상 답변을 기다리는 것이 의미 없다고 생각한 본인은 모 커뮤니티에 이 사실을 알렸다  이후 이슈화가 되자 병무청은 설치는 되었으나 운영은 하지 않았다는 대답을 내놓았지만 이에 대한 조사 경과 발표나 사과  책임 있는 대응은 전혀 보여주고 있지 않다      이는 개인의 자유와 권리의 본질적 내용을 침해하는 강제징집  국제노동기구가 인정한 강제징용 강제노동을 당하는 20대 남성들이 국가에 의해 신체를 감시당한 사건이다  한국 정부는 병역의무자에게는 인권이 없다고 생각하는가      언제까지 국가는 개인들을 노예 취급할 것인가  몇백원도 안되는 시급을 주며 강제로 폭력 속에서 부리는 것도 모자라 이제는 이런 어처구니 없는 일까지 일어나는 것에 경악하지 않을 수 없다       이에 본인은 위 사실을 강력히 규탄하며 아래와 같은 대응을 요구한다      1  서울지방병무청 징병검사소 탈의실에     가 설치된 경위와 운영 여부  관리 내역 등에 대해 낱낱이 조사하여 그 진상을 밝혀라 

In [5]:
df.loc[0]['title']

'서울지방병무청 탈의실에 설치된     에 대한 진상규명을 요구한다  또한 인권위의 미온적 대응을 규탄한다 '

# 2. 토크나이징 및 변수 생성

### - 1. 토크나이징

In [10]:
!pip install -r requirements.txt

  Using cached absl_py-0.11.0-py3-none-any.whl (127 kB)
  Using cached argon2_cffi-20.1.0-cp35-abi3-manylinux1_x86_64.whl (97 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached beautifulsoup4-4.6.0-py3-none-any.whl (86 kB)
  Using cached bleach-3.2.1-py2.py3-none-any.whl (145 kB)
  Using cached cached_property-1.5.2-py2.py3-none-any.whl (7.6 kB)
  Using cached cachetools-4.2.0-py3-none-any.whl (12 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached cffi-1.14.4-cp39-cp39-manylinux1_x86_64.whl (405 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached Cython-0.29.14.tar.gz (2.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 k

  Using cached pytz-2020.5-py2.py3-none-any.whl (510 kB)
  Using cached pywinpty-0.5.7.tar.gz (49 kB)
  Preparing metadata (setup.py) ... done
  Using cached PyYAML-5.3.1.tar.gz (269 kB)
  Preparing metadata (setup.py) ... done
  Using cached pyzmq-20.0.0-cp39-cp39-manylinux1_x86_64.whl (1.1 MB)
  Using cached qtconsole-5.0.1-py3-none-any.whl (118 kB)
  Using cached QtPy-1.9.0-py2.py3-none-any.whl (54 kB)
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached rsa-4.6-py3-none-any.whl (47 kB)
  Using cached scikit-learn-0.20.3.tar.gz (11.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached scipy-1.5.4-cp39-cp39-manylinux1_x86_64.whl (25.8 MB)
  Using cached Send2Trash-1.5.0-py3-none-any.whl (12 kB)
  Using cached sentencepiece-0.1.94-cp39-cp39-manylinux2014_x86_64.whl (1.1 MB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Preparing metada

  Created wheel for pyrsistent: filename=pyrsistent-0.17.3-cp39-cp39-linux_x86_64.whl size=70171 sha256=e0300e853f081fdc6c0ddc518e1b8c012aab2a369eae6896c51cf3e8408ff79a
  Stored in directory: /home/rkoh/.cache/pip/wheels/00/cc/12/e712d3979a4aeee423b34a832d97c803e1a67659060315a7e0
  Created wheel for pywinpty: filename=pywinpty-0.5.7-py3-none-any.whl size=10365 sha256=3f67c784c052ad4b0ac20316ecc0b9a827e0ebda9e8936db665d8ea3f60a4041
  Stored in directory: /home/rkoh/.cache/pip/wheels/53/76/53/01b01d6a997218adc05f673cd87078b0f3d5fbbe5b9ed7103b
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp39-cp39-linux_x86_64.whl size=44635 sha256=b4558134e7b02f21a081b06b8b6b27422a5b5cb397d00f7464f58b8471917049
  Stored in directory: /home/rkoh/.cache/pip/wheels/69/60/81/5cd74b8ee068fbe9e04ca0d53148f28f5c6e2c5b177d5dd622
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1517 lines of output]
      Partial import of sklearn duri

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=1b99b444d6cc6d6bd36c3b079bc4683bac4bc5d6f3a0bbf114d59b200c4f1028
  Stored in directory: /home/rkoh/.cache/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
  Created wheel for smart-open: filename=smart_open-4.0.1-py3-none-any.whl size=108257 sha256=12e0c43d2075b441cd3640b2fc23ae9778f3019f2b7299c336a4972cbfb30b0f
  Stored in directory: /home/rkoh/.cache/pip/wheels/7f/56/78/0bd642bc41a9f6f33ebbe6a9bfc2dcbbea7a5b6245ba43b850
Successfully built Cython future gensim matplotlib pandocfilters pyrsistent pywinpty PyYAML sklearn smart-open
Failed to build scikit-learn
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
  Attempting uninstall: testpath
    Found existing installation: testpath 0.6.0
    Uninstalling testpath-0.6.0:
      Successf

  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.9.0
    Uninstalling scipy-1.9.0:
      Successfully uninstalled scipy-1.9.0
  Attempting uninstall: requests
    Found existing installation: requests 2.28.1
    Uninstalling requests-2.28.1:
      Successfully uninstalled requests-2.28.1
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dateutil-2.8.2:
      Successfully uninstalled python-dateutil-2.8.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.4
    Uninstalling protobuf-3.19.4:
      Successfully uninstalled protobuf-3.19.4
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.20
    Uninstalling prompt-toolkit-3.0.20:
      Successfully uninstalled prompt-toolkit-3.0.20


      copying sklearn/externals/joblib/externals/loky/backend/semaphore_tracker.py -> build/lib.linux-x86_64-3.9/sklearn/externals/joblib/externals/loky/backend
      copying sklearn/externals/joblib/externals/loky/backend/reduction.py -> build/lib.linux-x86_64-3.9/sklearn/externals/joblib/externals/loky/backend
      copying sklearn/externals/joblib/externals/loky/backend/process.py -> build/lib.linux-x86_64-3.9/sklearn/externals/joblib/externals/loky/backend
      copying sklearn/externals/joblib/externals/loky/backend/_posix_reduction.py -> build/lib.linux-x86_64-3.9/sklearn/externals/joblib/externals/loky/backend
      copying sklearn/externals/joblib/externals/loky/backend/semlock.py -> build/lib.linux-x86_64-3.9/sklearn/externals/joblib/externals/loky/backend
      copying sklearn/externals/joblib/externals/loky/backend/compat_posix.py -> build/lib.linux-x86_64-3.9/sklearn/externals/joblib/externals/loky/backend
      copying sklearn/externals/joblib/externals/loky/backend/_posix

In [12]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 44.3 MB/s eta 0:00:00m eta 0:00:010:00:01


In [13]:
# konlpy.tag 중 한국어를 지원하는 모듈 : Okt(twitter), Kkma, Komoran, Mecab 등
from konlpy.tag import Okt

okt = Okt()

# 제목을 형태소 단위로 토크나이징
df['title_token'] = df.title.apply(okt.morphs)
# 내용을 명사 단위로 토크나이징 : 형태소 단위로 분리하면 양이 너무 많아져 비효율적임
df['content_token'] = df.content.apply(okt.nouns)

### - 2. 파생변수 생성

In [14]:
# 형태소 단위로 나누어진 청원 제목과 청원 내용에서 등장한 명사들을 합해 'token_final'에 저장
df['token_final'] = df.title_token + df.content_token

# 참여 인원은 천 단위마다 , 가 있어서(1,000) object로 인식
# , 를 없애 int형으로 바꿔줌
df['count'] = df['count'].replace({',' : ''}, regex = True).apply(lambda x : int(x))

# 잘 바뀌었는지 확인
print(df.dtypes)

# 청원 참여인원이 1000명 이상이면 yes 아니면 no
df['label'] = df['count'].apply(lambda x : 'yes' if x >= 1000 else 'No')

category         object
content          object
count             int64
end              object
start            object
title            object
title_token      object
content_token    object
token_final      object
dtype: object


In [15]:
# 라벨이 잘 붙었는지 확인
df.head(5)

,category,content,count,end,start,title,title_token,content_token,token_final,label
0,인권/성평등,본인은 2019년 8월 경 서울지방병무청 제1검사장 탈의실에서 믿을 수 없는 것을 ...,267,2020-02-01,2020-01-02,서울지방병무청 탈의실에 설치된 에 대한 진상규명을 요구한다 또한 인권위의 ...,"[서울, 지방, 병무청, 탈의실, 에, 설치, 된, 에, 대한, 진상, 규명, 을,...","[본인, 경, 서울, 지방, 병무청, 제, 검사, 탈의실, 수, 것, 발견, 탈의실...","[서울, 지방, 병무청, 탈의실, 에, 설치, 된, 에, 대한, 진상, 규명, 을,...",No
1,경제민주화,우리 나라 코스피 시총이 미국 애플보다 작다는 설이 돌 정도로 한국의 주식시장은 저...,271,2020-02-01,2020-01-02,주식시장 활성화 및 소액 개미 투자자 보호,"[주식시장, 활성화, 및, 소액, 개미, 투자자, 보호]","[우리, 나라, 코스피, 총, 미국, 애플, 설, 돌, 정도, 한국, 주식시장, 평...","[주식시장, 활성화, 및, 소액, 개미, 투자자, 보호, 우리, 나라, 코스피, 총...",No
2,행정,억울한 일로 국민청원을 신청합니다 저는 구치소 교도관입니다 ...,198,2020-02-01,2020-01-02,교정기관의 민낮,"[교정, 기관, 의, 민낮]","[일로, 국민, 청원, 신청, 저, 구치소, 교도관, 이주, 전, 구치소, 독, 거...","[교정, 기관, 의, 민낮, 일로, 국민, 청원, 신청, 저, 구치소, 교도관, 이...",No
3,안전/환경,미세먼지의 심각성은 이제 적극적인 대안을 요구 하고 있습니다 우리 일상에서 가...,170,2020-02-01,2020-01-02,미세먼지 저감 대책,"[미세먼지, 저, 감, 대책]","[미세먼지, 심각, 성은, 이제, 적극, 대안, 요구, 우리, 일상, 가장, 적극,...","[미세먼지, 저, 감, 대책, 미세먼지, 심각, 성은, 이제, 적극, 대안, 요구,...",No
4,교통/건축/국토,저는 우선 아이셋의 부모입니다 식구가 많은편이고 아이들이 성장함에 따라 집이 ...,2127,2020-02-01,2020-01-02,악질세입자 방지를 위한 세입자보호법을 재정해주세요,"[악질, 세, 입자, 방지, 를, 위, 한, 세, 입자, 보호, 법, 을, 재정, ...","[저, 우선, 아이, 셋, 부모, 식구, 편이, 아이, 집, 나름, 꿈, 가지, 좀...","[악질, 세, 입자, 방지, 를, 위, 한, 세, 입자, 보호, 법, 을, 재정, ...",yes


In [16]:
# 분석에 필요한 것만 추출하여 저장
df_drop = df[['token_final', 'label']]
df_drop.head(5)

,token_final,label
0,"[서울, 지방, 병무청, 탈의실, 에, 설치, 된, 에, 대한, 진상, 규명, 을,...",No
1,"[주식시장, 활성화, 및, 소액, 개미, 투자자, 보호, 우리, 나라, 코스피, 총...",No
2,"[교정, 기관, 의, 민낮, 일로, 국민, 청원, 신청, 저, 구치소, 교도관, 이...",No
3,"[미세먼지, 저, 감, 대책, 미세먼지, 심각, 성은, 이제, 적극, 대안, 요구,...",No
4,"[악질, 세, 입자, 방지, 를, 위, 한, 세, 입자, 보호, 법, 을, 재정, ...",yes


# 3. 단어 임베딩

### - 1. 단어 임베딩

In [18]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 40.4 MB/s eta 0:00:00m eta 0:00:010:01:01


In [19]:
# Word2Vec, Doc2Vec, FastText, LDA Model 등이 있음
from gensim.models import Word2Vec

# 임베딩 벡터를 생성할 대상이되는 데이터
embedding_model = Word2Vec(df_drop['token_final'],
                          # 1: Skip-Gram, 0: CBOW
                          sg = 1,
                          # 임베딩 벡터의 크기(Demension) 지정
                          vector_size = 100,
                          # 임베딩 벡터 생성 시 문맥 파악을 위해 고려해야 할 앞,뒤 토큰 수
                          window = 2,
                          # 전체 토큰에서 일정 횟수 이상등장하지 않는 토큰 제외
                          min_count = 1,
                          # 실행할 병렬 프로세서의 수 지정. 보통 4~6
                          workers = 4)
# 토큰 확인
print(embedding_model)

# 음주운전 과 유사한 단어 벡터 값 저장
model_result = embedding_model.wv.most_similar('음주운전')
print(model_result)

Word2Vec<vocab=42642, vector_size=100, alpha=0.025>
[('음주', 0.875351071357727), ('무면허', 0.8309951424598694), ('뺑소니', 0.8185737133026123), ('살인자', 0.7573997974395752), ('살인죄', 0.7570041418075562), ('전과자', 0.7518531680107117), ('형량', 0.7452690601348877), ('운전자', 0.7402490377426147), ('촉법소년', 0.7270837426185608), ('보호처분', 0.7255321741104126)]


### - 2. 임베딩 모델 저장 및 로드

In [21]:
# 임베딩 모델을 불러오기 위한 클래스
from gensim.models import KeyedVectors

# 임베딩 모델을 로컬 data폴더에 저장
embedding_model.wv.save_word2vec_format('data/petitions_tokens_w2v') # 모델 저장

# 로컬 data 폴더에 저장되어 있는 임베딩 모델 불러오기
loaded_model = KeyedVectors.load_word2vec_format('data/petitions_tokens_w2v') # 모델 로드

# 모델이 이상 없이 로드되었는지 확인
model_result = loaded_model.most_similar("음주운전")
print(model_result)

[('음주', 0.875351071357727), ('무면허', 0.8309951424598694), ('뺑소니', 0.8185737133026123), ('살인자', 0.7573997974395752), ('살인죄', 0.7570041418075562), ('전과자', 0.7518531680107117), ('형량', 0.7452690601348877), ('운전자', 0.7402490377426147), ('촉법소년', 0.7270837426185608), ('보호처분', 0.7255321741104126)]


# 4. 실험 설계

### - 1. 데이터셋 분할 및 저장

In [22]:
from numpy.random import RandomState

rng = RandomState()

# 전체 데이터 중 train 80%로 지정
tr = df_drop.sample(frac = 0.8, random_state = rng)
# 나머지 validation 지정
val = df_drop.loc[~df_drop.index.isin(tr.index)]

# 분할 결과 저장
tr.to_csv('data/train.csv', index = False, encoding = 'utf-8-sig')
val.to_csv('data/validation.csv', index = False, encoding = 'utf-8-sig')

### - 2. Field 클래스 정의

In [1]:
# pytorch에서 텍스트 데이터 처리 기능 및 텍스트 데이터를 제공해 자연어 처리를 지원하는 라브러리
import torchtext
# 텍스트에 대한 DataLoader, 추상화 기능 및 Iterator 제공
from torchtext.data import Field

# 토크나이저 정의
def tokenizer(text):
    # 변경하지 않으면 token_final의 ['토큰1', '토큰2'...] 자체를 하나의 토큰으로 인식해버림 : 리스트를 벗김
    # 변경 전 : ['토큰1', '토큰2'...] -> 변경 후 : 토큰1, 토큰2...
    text = re.sub('[\[\]\']', '', str(text))
    # 문자열을 , 단위로 나눠 분리한 결과 반환
    text = text.split(',')
    return text

'''
fix_length : sentence의 길이 제한
tokenize : tokenize 설정 옵션(default : string.split() 띄어쓰기)
lower : sentence를 모두 소문자로(default :True)
sequential : 순서가 있는 데이터인지 여부 옵션. TXT는 순서가 있는 데이터임(default : True)
pad_first : fix_length 대비 짧은 문장의 경우 padding을 해야하는데, padding을 앞에서 줄 것인지 여부(default : False)
pad_token : pad_first에 대한 특수 토큰 설정(default : "")
use_vocab : 단어장 객체를 사용할 것인지 여부(default : True)
'''
TEXT = Field(tokenize = tokenizer)
# 딥러닝 모델의 output임. sequential 옵션을 False로 두는 이유는 label데이터는 순서가 불필요한 yes or no 형식이기 때문
LABEL = Field(sequential = False)

OSError: [WinError 127] 지정된 프로시저를 찾을 수 없습니다

### - 3. 데이터 불러오기

In [2]:
# 파일에서 데이터를 읽어 데이터셋의 생성 지원
from torchtext.data import TabularDataset


train, validation = TabularDataset.splits(
    # 데이터의 경로 지정
    path = 'data/',
    # 각 train, validation set의 파일 이름 지정
    train = 'train.csv',
    validation = 'validation.csv',
    # csv, tsv, json에 해당하는 데이터 포맷 지정
    format = 'csv',
    # 앞에서 설정한 field 지정
    fields = [('text', TEXT), ('label', LABEL)],
    # skip_header : 데이터의 첫 행이 컬럼명인 경우 True
    skip_header = True
)

print("Train : ", train[0].text, train[0].label)
print("Validation : ", validation[0].text, validation[0].label)

OSError: [WinError 127] 지정된 프로시저를 찾을 수 없습니다

### - 4. 단어장 및 DataLoader정의

In [28]:
import torch
# 임베딩 벡터를 생성하기 위한 클래스
from torchtext.vocab import Vectors
# 데이터셋에서 배치 사이즈만큼 데이터를 로드하는 Iterator 지원
from torchtext.data import BucketIterator

# 사전에 훈련된 petitions_tokens_w2v 임베딩 벡터 저장
vectors = Vectors(name = "data/petitions_tokens_w2v")

# vectors : Pre-Train Vector를 가져와 단어장에 세팅하는 옵션. 각 단어의 임베딩 벡터 값을 petitions_tokens_w2v 값들로 초기화
# min_freq : 등장 빈도가 일정 수준 이상인 토큰만 허용
# max_size : 전체 vocab size에 제한을 둠
TEXT.build_vocab(train, vectors = vectors, min_freq = 1, max_size = None)

LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# datasets : 로드한 데이터셋 지정
# batch_size : 각 batch data별 크기를 지정
# device : Batch 생성 장비를 지정
# sort : 전체 데이터 정렬 여부 선택(default : false)
train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8,
    device = device,
    sort = False
)

print("임베딩 벡터의 개수와 차원 : {}".format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원 : torch.Size([41340, 100])


# 5. TextCNN 모델 설계

### - 1. TextCNN 모델링

In [29]:
# 신경망을 쌓기 위한 모듈
# convolution, fully connected, dropout 등 제공
import torch.nn as nn
# 모델 최적화(optimization) 알고리즘 제공
import torch.optim as optim
# torch.nn의 모든 함수가 포함되어 있는 모듈
import torch.nn.functional as F

# 딥러닝 모델 관련 기본 함수를 포함하고 있는 nn.Module 클래스를 상속받는 TextCNN 클래스 정의
class TextCNN(nn.Module):
    # TextCNN 클래스로 생성한 객체를 초기화하는 역할
    # vocab_bulit : train데이터로 생성한 단어장을 인자로 받음
    # emb_dim : 임베딩 벡터의 dimension(크기)을 인자로 받음(100)
    # dim_channel : Feature Map 이후로 생성되는 채널의 수를 인자로 받음(100)
    # kernel_wins : filter의 크기를 리스트 형태로 받음([3, 4, 5])
    # num_class : output의 클래스 개수를 받음(2)
    def __init__(self, vocab_built,emb_dim, dim_channel, kernel_wins, num_class):
        # nn.Module 내에 있는 메서드를 상속받아 이용합니다
        super(TextCNN, self).__init__()
        # 임베딩 설정을 위해서 Vocab Size * Embedding Dimension (41340 * 100) 크기의 행렬을 만듦
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        # 앞서 Word2Vec으로 학습한 임베딩 벡터 값들을 가져옵니다
        self.embed.weight.data.copy_(vocab_built.vectors)
        # nn.Conv2d 함수에 임베딩 결과를 전달해 Filter를 생성합니다
        # 1 : input의 채널은 1이므로
        # dim_channel : output의 채널은 인자로 받은 dim_channel 크기와 같음(10)
        # w : filter의 크기. kernel_wins가 [3, 4, 5]이므로 w는 각각 3, 4, 5값을 가짐
        # emb_dim : 임베딩 벡터의 Dimension(크기)임 (100)
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        # Activation Map을 생성하는 Activation Function인 ReLU 정의
        self.relu = nn.ReLU()
        # 오버피팅을 방지하기 위해 일부 Node의 가중 값을 계산하지 않는 Dropout 적용
        # 가중치 행렬에서 랜덤하게 40%에 0을 넣어 연산
        self.dropout = nn.Dropout(0, 4)
        # 클래스에 대한 Score를 생성하기 위해 len(kernel_wins) * dim_channel, num_class(3*10, 2) 크기의 Fully Connected Layer 생성
        self.fc = nn.Linear(len(kernel_wins) * dim_channel, num_class)
    
    # TextCNN의 Foreard Propagation을 정의
    # 설계한 모델에 데이터를 입력했을 때 Output을 계산하기까지의 과정
    def forward(self, x):
        # 인자로 받은 x의 임베딩 정보 전달
        emb_x = self.embed(x)
        # unsqueeze(1)을 통해 emb_x의 첫 번째 축(축 1)에 차원 추가
        # (Batch_Size * 41340 * 100) -> (Batch Size * 1 * 41340 * 100)
        # 2차원의 텍스트 데이터를 모델에 이미지처럼 입력하려면 차원을 추가하여 3차원 형태로 변환해야 하기 때문
        emb_x = emb_x.unsqueeze(1)
        # self.convs에는 Filter 세개([3 * 100, 4 * 100, 5 * 100])가 리스트 형태로 이루어짐
        # 리스트의 각 원소에 대한 output으로 feature map을 생성하여, 
        # 세 가지 filter를 각각 통과한 결과인 feature map 3개가 con_x에 리스트 형태로 저장됨
        # 이후에 Relu 함수를 통과해 Activation Map을 생성
        con_x = [self.relu(conv(emb_x)) for conv in self.convs]
        # Max Pooling을 진행해 리스트 형태로 이루어진 
        # Pooling Layer([dim_channel(10) * 1], [dim_channel(10) * 1], [dim_channel(10) * 1]) 생성
        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]
        # Pooling Layer 크기(10) * 1 형태로 이루어진 1차원 Pooling 벡터 세 개를 각각 Concat하여
        # 하나의 Fully Connected Layer(Pooling Layer 크기(10) * 3) 생성
        fc_x = torch.cat(pool_x, dim = 1)
        # Pooling Layer 크기(10) * 3 형태의 Fully Connected Layer에 차원을 줄여 (30 * 1) 형태인 최종 Fully Connected Layer 생성
        fc_x = fc_x.squeeze(-1)
        # Fully Connected Layer에 Dropout 적용
        fc_x = self.dropout(fc_x)
        
        # (30 * 1) 크기ㅣ fc_x를 fc 함수에 통과시켜 (1 * 2) 크기인 Fully Connected Layer의 Propagation Logit 값 연산
        logit = self.fc(fc_x)
        
        return logit

### - 2. 모델 학습 함수 정의

In [38]:
def train(model, device, train_itr, optimizer):
    # TextCNN모델을 Train Mode로 변경하여 Parameter를 업데이트 가능하도록 함
    # def train함수와는 다른 의미
    model.train()
    # 학습 과정에서 모델이 올바른 클래스로 분류한 경우에 대한 Accuracy와 Loss를 계산하기 위해
    # corrects와 train_loss를 0으로 설정
    corrects, train_loss = 0.0, 0
    
    for batch in train_itr:
        # 미니 배치 단위로 저장되어 있는 텍스트 데이터와 레이블 데이터를 text, target에 저장
        text, target = batch.text, batch.label
        # 연산을 위해 텍스트 데이터를 역행렬로 변환
        text = torch.transpose(text, 0, 1)
        # target의 각 값을 1씩 줄임
        target.data.sub_(1)
        # 미니 배치 내에 있는 데이터를 이용해 모델을 학습시키고자 장비에 할당
        text, target = text.to(device), target.to(device)
        # 이전 미니 배치에 있는 텍스트 데이터와 레이블 데이터를 바탕으로 계산된
        # Loss의 Gradient값이 Optimizer에 할당돼 있으므로 Optimizer의 Gradient 값 초기화
        optimizer.zero_grad()
        # 텍스트 데이터를 TextCNN 모델의 Input으로 이용해 output 계산
        logit = model(text)
        # TextCNN 모델에서 Return 받은 Logit 값에 Softmax 함수를 통과시켜 Yes or No로 분류
        # 예측값과 실제 레이블 데이터를 비교하여 Negative Log Loss 값 계산
        # Softmax와 Negative Log Loss 값은 torch.nn.functional 모듈의 cross_entropy 함수로 동시 연산
        loss = F.cross_entropy(logit, target)
        # Forward를 통해서 알고 있는 Computational Graph를 거꾸로 올라가며 Loss 값을 가지며
        # Parameter에 대해 미분하면서 누적된 Gradient를 구함
        loss.backward()
        # loss.backward()를 통해 각 Parameter별로 할당된 Gradient 값을 이용해 Parameter 값을 업데이트
        optimizer.step()
        # 현재 진행 중인 미니 배치에서의 loss 값을 train_loss에 누적하여더함
        train_loss += loss.item()
        # 인덱스별로 계산된 logit 값에서 더 큰 확률을 가진 클래스 저장
        result = torch.max(logit, 1)[1]
        # TextCNN 모델의 예측값과 레이블 데이터를 비교해 맞으면 corrects에 더해 올바르게 예측한 횟수 저ㅇ
        corrects += (result.view(target.size()).data == target.data).sum()
    # 현재까지 계산된 train_loss 값을 train_itr 내에 있는 미니 배치 개수ㄴ큼 나눠 train_itr의 평균 Loss값 계산
    train_loss /= len(train_itr.dataset)
    # train_itr 데이터 중 얼마나 맞혔는지 정확도 계산
    accuracy = 100.0 * corrects / len(train_itr.dataset)
    # 계산된 loss값과 accuracy 값 반환
    return train_loss, accuracy
                                   
                                

### - 3. 모델 평가 함수 정의

In [39]:
# train 코드와 동일함
def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0, 0
    
    for batch in itr:
        
        text = batch.text
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)
        
        test_loss += loss.item()
        result = torch.max(logit, 1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()
        
    test_loss /= len(itr.dataset)
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

### - 4. 모델 학습 및 성능 확인

In [41]:
model = TextCNN(vocab, 100, 10, [3, 4, 5], 2).to(device)
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Back Propagation을 통해 Parameter를 업데이트할 때 이용하는 Optimizer를 정의함
# Adam 알고리즘을 사용하며 Parameter 업데이트 시 Learning Rate를 0.001로 지정
optimizer = optim.Adam(model.parameters(), lr = 0.001)

best_test_acc = -1

for epoch in range(1, 3+1):
    # 정의한 Train 함수 실행
    # 모델은 TextCNN, 학습데이터는 train_iter, Optimizer는 Adam
    tr_loss, tr_acc = train(model, device, train_iter, optimizer)
    # 각 epoch별로 출력되는 loss값과 accuracy값 출력
    print('Train Epoch : {} \t Loss : {} \t Accuracy : {}%'.format(epoch, tr_loss, tr_acc))
    # 정의한 evaluate 함수 실행
    # 모델은 TextCNN, 학습 데이터는 validation_iter로 설정하여 모델이 train 데이터를 잘 학습하는지 확인
    val_loss, val_acc = evaluate(model, device, validation_iter)
    
    print('Valid Epoch : {} \t Loss : {} \t Accuracy : {}%'.format(epoch, val_loss, val_acc))
    
    if val_acc > best_test_acc : 
        best_test_acc = val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")
        
    print('-'*30)

TextCNN(
  (embed): Embedding(41340, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0, inplace=4)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)
Train Epoch : 1 	 Loss : 0.07981724791626763 	 Accuracy : 63.75646209716797%
Valid Epoch : 1 	 Loss : 0.07542247911367346 	 Accuracy : 67.50919342041016%
model saves at 67.50919342041016 accuracy
------------------------------
Train Epoch : 2 	 Loss : 0.06798936495796008 	 Accuracy : 72.3836898803711%
Valid Epoch : 2 	 Loss : 0.07427166713237324 	 Accuracy : 68.42830657958984%
model saves at 68.42830657958984 accuracy
------------------------------
Train Epoch : 3 	 Loss : 0.04688623366275099 	 Accuracy : 83.97472381591797%
Valid Epoch : 3 	 Loss : 0.08604942225193714 	 Accuracy : 65.53308868408203%
------------------------------